In [1]:
# 导入 pandas 用于数据分析
import pandas as pd
# 利用 pandas 的 read_csv 模块直接从互联网收集泰坦尼克号乘客数据
titanic = pd.read_csv('http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic.txt')
# 观察前几行数据
# 数据种类各异，数值型，类别型，还有缺失数据
titanic.head()

,row.names,pclass,survived,name,age,embarked,home.dest,room,ticket,boat,sex
0,1,1st,1,"Allen, Miss Elisabeth Walton",29.0000,Southampton,"St Louis, MO",B-5,24160 L221,2,female
1,2,1st,0,"Allison, Miss Helen Loraine",2.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
2,3,1st,0,"Allison, Mr Hudson Joshua Creighton",30.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,(135),male
3,4,1st,0,"Allison, Mrs Hudson J.C. (Bessie Waldo Daniels)",25.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
4,5,1st,1,"Allison, Master Hudson Trevor",0.9167,Southampton,"Montreal, PQ / Chesterville, ON",C22,NaN,11,male


In [2]:
# 使用 pandas，数据都转入 pandas 独有的 dataframe 格式(二维数据表格), 直接使用info(),查看数据的统计特性
titanic.info()
# 共有 1313 条乘客信息，并且有些特征数据是完整的(pclass, name)，有些是缺失的，有些是数值类型的，有些则是字符串

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 11 columns):
row.names    1313 non-null int64
pclass       1313 non-null object
survived     1313 non-null int64
name         1313 non-null object
age          633 non-null float64
embarked     821 non-null object
home.dest    754 non-null object
room         77 non-null object
ticket       69 non-null object
boat         347 non-null object
sex          1313 non-null object
dtypes: float64(1), int64(2), object(8)
memory usage: 112.9+ KB


In [3]:
# 对数据进行预处理和分析
# 使用决策树模型预测泰坦尼克号乘客的生还情况

In [4]:
# 特征 的选择很重要 *****
# 根据对泰坦尼克号事故的了解，sex，age，pclass 这些特征都很有可能是决定幸免与否的关键因素
x = titanic[['pclass', 'age', 'sex']]
y = titanic['survived']
# 对当前选择的特征进行探查
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 3 columns):
pclass    1313 non-null object
age       633 non-null float64
sex       1313 non-null object
dtypes: float64(1), object(2)
memory usage: 30.9+ KB


In [5]:
# 通过查看输出，我们设计对如下几个数据的处理任务
# 1. age 数据列只有 633 个，需要补完整
# 2. sex 与 pclass 两个数据列的值都是类别型的，需要转化为数值特征，用 0 或者 1 代替

In [6]:
# 补全 age 里的数据，使用平均数或者中位数都是对模型偏离造成最小影响的策略
x['age'].fillna(x['age'].mean(), inplace = True)
# 对补完的数据重新查看
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 3 columns):
pclass    1313 non-null object
age       1313 non-null float64
sex       1313 non-null object
dtypes: float64(1), object(2)
memory usage: 30.9+ KB


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:3549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [7]:
# 数据分割
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 33)

In [9]:
# 使用 scikit-learn.feature_extraction 中的特征转化器
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer(sparse = False)

DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
        sparse=False)


In [10]:
# 转换特征后，我们发现凡是类别型的特征都单独剥离出来，独成一列特征，数值型的则保持不变
x_train = vec.fit_transform(x_train.to_dict(orient = 'record'))
print(vec.feature_names_)

['age', 'pclass=1st', 'pclass=2nd', 'pclass=3rd', 'sex=female', 'sex=male']


In [11]:
# 同样需要对测试数据的特征进行转换
x_test = vec.transform(x_test.to_dict(orient='record'))

In [12]:
# 从 sklearn.tree 中导入决策树分类器
from sklearn.tree import DecisionTreeClassifier
# 使用默认配置初始化决策树分类器
dtc = DecisionTreeClassifier()
# 使用分割到的训练数据进行模型学习
dtc.fit(x_train, y_train)
# 用训练好的决策树模型对测试特征数据进行预测
y_predict = dtc.predict(x_test)

In [13]:
# 性能测评 => 使用同样用于分类任务的多种性能测评指标，对乘客是否生还测预测结果进行评价

In [14]:
# 使用准确率，召回率，正确率以及F1指标，这4个测试决策树模型对泰坦尼克号乘客是否生还的预测性能
# 使用模型自带的评估函数进行准确性测评
# 决策树模型对泰坦尼克号乘客是否生还的预测性能
# 导入 classification_report 
from sklearn.metrics import classification_report
# 输出预测准确性
print(dtc.score(x_test, y_test))
# 输出更加详细的分类性能
print(classification_report(y_predict, y_test, target_names = ['died', 'survived']))

0.781155015198
             precision    recall  f1-score   support

       died       0.91      0.78      0.84       236
   survived       0.58      0.80      0.67        93

avg / total       0.81      0.78      0.79       329

